Comparison text generations from GPT and GPT-2 to illustrate the notion of a model being skewed.

In [1]:
from transformers import pipeline, set_seed

generation_gpt = pipeline("text-generation", model="openai-gpt")
generation_gpt2 = pipeline("text-generation", model="gpt2")

/home/ssanchez/env/transformers/lib/python3.8/site-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
/home/ssanchez/env/transformers/lib/python3.8/site-packages/torchvision/transforms/v2/__init__.py:54: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feed

config.json:   0%|          | 0.00/656 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/479M [00:00<?, ?B/s]

{"timestamp":"2025-07-01T14:46:16.894739Z","level":"WARN","fields":{"message":"Reqwest(reqwest::Error { kind: Request, url: \"https://cas-server.xethub.hf.co/reconstruction/986657f8b77a4a690d73500aa22cbda25b96a55bd9b689e8312ebd983a7108d5\", source: hyper_util::client::legacy::Error(Connect, ConnectError(\"dns error\", Custom { kind: Uncategorized, error: \"failed to lookup address information: Name or service not known\" })) }). Retrying..."},"filename":"/home/runner/work/xet-core/xet-core/cas_client/src/http_client.rs","line_number":200}
{"timestamp":"2025-07-01T14:46:16.894830Z","level":"WARN","fields":{"message":"Retry attempt #0. Sleeping 651.203428ms before the next attempt"},"filename":"/root/.cargo/registry/src/index.crates.io-1949cf8c6b5b557f/reqwest-retry-0.7.0/src/middleware.rs","line_number":171}


generation_config.json:   0%|          | 0.00/74.0 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/816k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/458k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.27M [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [2]:
def model_size(model):
    return sum(t.numel() for t in model.parameters())

print(f"GPT size:{model_size(generation_gpt.model)/1000**2:.1f}M parameters")
print(f"GPT2 size:{model_size(generation_gpt2.model)/1000**2:.1f}M parameters")

GPT size:116.5M parameters
GPT2 size:124.4M parameters


In [3]:
def enum_pipeline_outputs(pipe, prompt, num_return_sequences):
    out = pipe(prompt, num_return_sequences=num_return_sequences, clean_up_tokenization_spaces=True)
    return "\n".join(f"{i+1}." + s["generated_text"] for i, s in enumerate(out))

prompt = "\nWhen they came back"
print("GPT completions:\n" + enum_pipeline_outputs(generation_gpt, prompt, 3))
print("")
print("GPT2 completions:\n" + enum_pipeline_outputs(generation_gpt2, prompt, 3))

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


GPT completions:
1.
When they came back from the cemetery and said their good - byes at the gravesite. 
 at first i was so happy to have them return to me, i couldn't contain myself. with the relief we could share and the sense of relief they
2.
When they came back into the room. they were smiling at me and i thought i saw a hint of relief. we were all exhausted as we headed down the hallway and into the living room. 
 " where have you been? we've been so
3.
When they came back out to the clearing she was with, but they didn't see her. when she stopped by the river to drink from the spring, she saw a strange object floating across toward her. then, as suddenly as it had appeared,

GPT2 completions:
1.
When they came back, there was a small group of us waiting for them there as our security guard stood by them, then they began to run, kicking, slashing and trying to frighten me into silence. It took about ten minutes before they
2.
When they came back, the car was already moving.

"Ther

## Building a Custom Code Dataset
### Creating a dataset with Google BigQuery

tota la perafernalia que he hagut de fer per executar una consulta, guardar els resultats en una taula d'un dataset i passar les dades a una carpeta del bucket 'npl_transformers'.
Després he instalat el [gcloud CLI](https://cloud.google.com/sdk/docs/install) al servidor.
Llavors he pogut llançar la comana: ``gsutil -m -o "GSUtil:parellel_process_count=1" cp -r gs:<nom_del_bucket>/<carpeta>/resultados-*.json.gz /datasets/codeparrot``

## Working with Large Datasets

Loading a very large dataset is often a challenging task, in particular when the data is larger than your machine's RAM.
In our example, we have 50 GB of compressed data and about 200 GB of uncompressed data.

Thankfully Datasets has two specific features that allow you to set yourself free from RAM and hard drive space limitations: **memory mapping** and **streaming**.

### Memory mapping

Overcoming RAM limitations --> uses a mechanism of zero-copy and zero-overhead memory mapping.
Basically, each dataset is cached on the drive in a file that is a direct reflection of the content in RAM memory. Instead of loading the dataset in RAM, it opens a read-only pointer to this file and uses it as a substitue for RAM (using hard drive as an extension of RAM).

Here, we will direcly load our 50GB of compressed JSON files that we have stored locally in the *codeparrot* repository.
Decompress JSON files. Be careful, cause this uses **180GB of free disk!!!**

*delete_extracted=True* in the dataset's downloading configuration, we can make sure that we delete all the files we don't need anymore as soon as possible

In [17]:
from datasets import load_dataset, DownloadConfig, Features, Value
import os

download_config = DownloadConfig(delete_extracted=True)
dataset_path = os.path.expanduser("~/datasets/codeparrot/export_results/*.json.gz")

features = Features({
    "repo_name": Value("string"),
    "path": Value("string"),
    "copies": Value("string"),
    "size": Value("string"),
    "content": Value("string"),
    "license": Value("string")
})

dataset = load_dataset("json",
                       data_files=dataset_path, split="train",
                      download_config=download_config, features=features)

Resolving data files:   0%|          | 0/500 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Loading dataset shards:   0%|          | 0/206 [00:00<?, ?it/s]

In [8]:
# !gunzip -c ~/datasets/codeparrot/export_results/resultados-000000000000.json.gz > ./uno.json

In [9]:
#from datasets import load_dataset
#
#dataset = load_dataset("json", data_files="./uno.json", split="train")
#print(dataset)

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['repo_name', 'path', 'copies', 'size', 'content', 'license'],
    num_rows: 38184
})


In [18]:
import psutil
print(f"Number of python files code in dataset : {len(dataset)}")
ds_size = sum(os.stat(f["filename"]).st_size for f in dataset.cache_files)
# os.stat.st_size is expressed in bytes, so we convert to GB
print(f"Dataset size (cache file) : {ds_size / 2**30:.2f} GB")
# Process.memory_ingo is expressed in bytes, so we convert to MB
print(f"RAM used: {psutil.Process(os.getpid()).memory_info().rss >> 20} MB")

Number of python files code in dataset : 18923569
Dataset size (cache file) : 97.06 GB
RAM used: 1720 MB


What if you can't free enough disk space to store the full dataset locally?

## Streaming

An alternative to scaling up the server you are using is to *stream* the dataset.

In [19]:
streamed_dataset = load_dataset("json",
                       data_files=dataset_path, split="train",
                      download_config=download_config, features=features,
                               streaming=True)

Resolving data files:   0%|          | 0/500 [00:00<?, ?it/s]

In streaming mode, the compressed JSON files will be opened and read on the fly. Out dataset is now an *IterableDataset* object. This means that we cannot access random elements of it (streamed_dataset[1264]), but we need to read it in order, for instance with ``next(iter(streamed_dataset))``

In [20]:
iterator = iter(streamed_dataset)

print(dataset[0] == next(iterator))
print(dataset[1] == next(iterator))

True
True


The original raw files are extracted and read on the fly when a new batch of examples is requested, and onñy that batch is loaded in memory.

One step further-- instead of pointing to the local dataset we can reference the dataset on the Hub, and then directly download samples without downloading the raw files locally:

In [23]:
remote_dataset = load_dataset('transformersbook/codeparrot', split='train', streaming=True)

README.md: 0.00B [00:00, ?B/s]

Resolving data files:   0%|          | 0/184 [00:00<?, ?it/s]

In [5]:
from huggingface_hub import notebook_login

notebook_login()

In [8]:
! huggingface-cli repo create --repo-type dataset --organization sergi24sanchez codeparrot-train

The --type argument is deprecated and will be removed in a future version. Use --repo-type instead.
The --organization argument is deprecated and will be removed in a future version. Pass the organization namespace directly in the repo_id.
Successfully created sergi24sanchez/codeparrot-train on the Hub.
Your repo is now available at https://huggingface.co/datasets/sergi24sanchez/codeparrot-train


In [9]:
! huggingface-cli repo create --repo-type dataset sergi24sanchez/codeparrot-valid

The --organization argument is deprecated and will be removed in a future version. Pass the organization namespace directly in the repo_id.
Successfully created sergi24sanchez/codeparrot-valid on the Hub.
Your repo is now available at https://huggingface.co/datasets/sergi24sanchez/codeparrot-valid


# Builiding a Tokenizer

Now that we have gathered and loaded our large dataset. Let's process efficiently the data to feed to our model.

Previously: pretrained models --> pretrained tokenizers. Data passed through a specific preprocessing pipeline defined in the tokenizer.
*When using a pretreined model, it's important to stick with the same preprocessing design choices selected for pretraining*. Otherwise the model may be fed out-of-distribution patterns or unknown tokens.

Kinds of problems we might run into when using an existing tokenizer:

- T5 tokenizer has an extensive step of stopword filtering.
- CamemBERT only comprised French text

We can test these features:

In [2]:
from transformers import AutoTokenizer

def tok_list(tokenizer, string):
    input_ids = tokenizer(string, add_special_tokens=False)["input_ids"]
    return [tokenizer.decode(tok) for tok in input_ids]

tokenizer_t5 = AutoTokenizer.from_pretrained("t5-base")
tokenizer_camembert = AutoTokenizer.from_pretrained("camembert-base")

print(f"T5 tokend for 'sex': {tok_list(tokenizer_t5,'sex')}")
print(f"CamemBERT tokend for 'being': {tok_list(tokenizer_camembert,'being')}")

T5 tokend for 'sex': ['', 's', 'ex']
CamemBERT tokend for 'being': ['be', 'ing']


## The Tokenizer Model

Normalization, pretokenization, tokenizer model, and postprocessing.
The part of the tokenizer that can be trained is the *tokenizer model* (BPE, WordPiece, and Unigram)

## Measuring Tokenizer Performance

- *Subword fertility*, which calculates the average number of subwords produced per tokenizer word
- *Proportion of continued words*, refers to the proportion of tokenized words in the corpus that are split into at least two subtokens
- *Coverage metrics* like the proportion of unknown words or rarely used tokens in a tokenized corpus

Robustness to misspelling or noise is often estimated, as well as model performance on such out-of-domain examples.

The measures tend to ignore the interaction of the tokenizer with the model.

Generally, the performance of the tokenizer is best estimated by using the downstream performance of the model

## A Tokenizer for Python

If we split on whitespaces and remove them, we'll lose all the identation information.
Line breaks are not meaningful.
Splitting on punctuation might not make as much sense as it would in natural language.

In [3]:
python_code = r"""def say_hello()
    print("Hello, World!")

# Print it
say_hello()
"""
tokenizer = AutoTokenizer.from_pretrained("gpt2")
print(tokenizer(python_code).tokens())

['def', 'Ġsay', '_', 'hello', '()', 'Ċ', 'Ġ', 'Ġ', 'Ġ', 'Ġprint', '("', 'Hello', ',', 'ĠWorld', '!"', ')', 'Ċ', 'Ċ', '#', 'ĠPrint', 'Ġit', 'Ċ', 'say', '_', 'hello', '()', 'Ċ']


In [11]:
# Let's see what normalization is applied in the tokenizer
print(tokenizer.backend_tokenizer.normalizer)

None


GPT2 tokenizer works directly on the raw Unicode.
Pretokenization:

In [12]:
print(tokenizer.backend_tokenizer.pre_tokenizer.pre_tokenize_str(python_code))

[('def', (0, 3)), ('Ġsay', (3, 7)), ('_', (7, 8)), ('hello', (8, 13)), ('()', (13, 15)), ('ĊĠĠĠ', (15, 19)), ('Ġprint', (19, 25)), ('("', (25, 27)), ('Hello', (27, 32)), (',', (32, 33)), ('ĠWorld', (33, 39)), ('!")', (39, 42)), ('Ċ', (42, 43)), ('Ċ', (43, 44)), ('#', (44, 45)), ('ĠPrint', (45, 51)), ('Ġit', (51, 54)), ('Ċ', (54, 55)), ('say', (55, 58)), ('_', (58, 59)), ('hello', (59, 64)), ('()', (64, 66)), ('Ċ', (66, 67))]


Ġ symbols?? And what are the numbers accompanying the tokens?

Tokenizers has a very useful feature for switching between sstrings and tokens, *offset tracking*. These numbers simply indicate where in the original string each tokens comes from.

Odd-looking characters. *Byte-level* means that this tokenizer works on bytes instead of Unicode characters. Each Unicode character is composed of between 1-4 bytes. While there are 143,859 Unicode characters, there are only 256 elements in the byte alphabet. If we work on bytes we can thus express al the strings composed from the UTF-8 world as longer strings in this alphabet of 256 values.
That is, we can have a **model using an alphabet of only 256 words and be able to process any Unicode string**.

In [13]:
a, e = u"a", u"€"
byte = ord(a.encode("utf-8"))
print(f"'{a}' is encoded as '{a.encode('utf-8')}' with a single byte: {byte}'")
byte = [ord(chr(i)) for i in e.encode('utf-8')]
print(f"'{e}' is encoded as '{e.encode('utf-8')}' with bytes: {byte}'")

'a' is encoded as 'b'a'' with a single byte: 97'
'€' is encoded as 'b'\xe2\x82\xac'' with bytes: [226, 130, 172]'


Why work on a byte level?

Middle-ground solution: construct a medium-sized vocabulary by extending the 256-word vocabulary + most common combinations of bytes.

BPE algorithm: progressively construct a vocabulary of a predefined size by creating new vocabulary tokens through iteratibely merfin the mos frequently cooccurring pair of tokens in the vocabulary.

In [4]:
# entire mapping of 256 elementary values to Unicode strings
from transformers.models.gpt2.tokenization_gpt2 import bytes_to_unicode

bytes_to_unicode_map = bytes_to_unicode()
unicode_to_byte_map = dict((v, k) for k,v in bytes_to_unicode_map.items())
base_vocab = list(unicode_to_byte_map.keys())

print(f"Size of base vocab: {len(base_vocab)}")
print(f"First element: '{base_vocab[0]}', last element: '{base_vocab[-1]}''")

Size of base vocab: 256
First element: '!', last element: 'Ń''


BPE algorithms are typically designed to work on characters.

In [8]:
print(f"size of the vocab: {len(tokenizer)}")

size of the vocab: 50257


In [14]:
print(tokenizer(python_code).tokens())

['def', 'Ġsay', '_', 'hello', '()', 'Ċ', 'Ġ', 'Ġ', 'Ġ', 'Ġprint', '("', 'Hello', ',', 'ĠWorld', '!"', ')', 'Ċ', 'Ċ', '#', 'ĠPrint', 'Ġit', 'Ċ', 'say', '_', 'hello', '()', 'Ċ']


BPE tokenizer keeps most of the words but will split the mutiple spaces of our indentation into several consecutive spaces.
This happens because this tokenizer is trained on texts where consecutive spaces are rare. The BPE model thus doesn't include a specific token in the vocabulary for indentation.

**Clear case where the tokenizer is poorly suited fot the dataset's domain**

-------> SOLUTION: retrain the tokenizer on the target corpus

## Training a Tokenizer

In [9]:
tokens = sorted(tokenizer.vocab.items(), key=lambda x: len(x[0]), reverse=True)
print([f'{tokenizer.convert_tokens_to_string([t])}' for t,_ in tokens[:8]]);

['ÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂ', ' =================================================================', ' ----------------------------------------------------------------', '________________________________________________________________', '----------------------------------------------------------------', 'ÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂ', '................................................................', '================================================================']


These tokens look like separator lines that are likely to be used on forums.

Now, let's look at the last words added to the vocabulary, and thus the least frequent:

In [11]:
tokens = sorted(tokenizer.vocab.items(), key=lambda x: x[1], reverse=True)
print([f'{tokenizer.convert_tokens_to_string([t])}' for t,_ in tokens[:12]]);

['<|endoftext|>', ' gazed', ' informants', ' Collider', ' regress', 'ominated', ' amplification', 'Compar', '…."', ' (/', 'Commission', ' Hitman']


For each of these tokens our model will have to learn an associated word embedding, and we don't want the embedding matrix to contain too many noisy words.

Note the very time- and space-specific words, already embedded at a low level in our modeling approach. Such specific tokens by a BPE tokenizer can also be an indication that the target vocabulary size is too larfe or that the corpus contains idiosyncratic tokens.

Let's train a fresh tokenizer and anayse its learned vocabulary.

In [13]:
# Let's select 1-2GB of data (about 100K documents)
from tqdm.auto import tqdm
from datasets import load_dataset

length = 100000
dataset_name = "sergi24sanchez/codeparrot-train"
dataset = load_dataset(dataset_name, split="train", streaming=True)
iter_dataset = iter(dataset)

def batch_iterator(batch_size=10):
    for _ in tqdm(range(0, length, batch_size)):
        yield [next(iter_dataset)['content'] for _ in range(batch_size)]

new_tokenizer = tokenizer.train_new_from_iterator(batch_iterator(), vocab_size=12500, initial_alphabet=base_vocab)

Resolving data files:   0%|          | 0/499 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

In [23]:
tokens = sorted(new_tokenizer.vocab.items(), key=lambda x: x[1], reverse=False)
print(len(new_tokenizer.vocab))
print([f'{new_tokenizer.convert_tokens_to_string([t])}' for t,_ in tokens[257:280]]);

12500
['  ', '    ', '   ', '\n    ', 'se', 'in', 're', 'on', 'te', 'or', '\n       ', 'st', 'de', '\n   ', '        ', 'th', ' =', 'le', ' a', 'lf', 'ti', 'me', 'self']


In [24]:
print([f"{new_tokenizer.convert_tokens_to_string([t])}" for t,_ in tokens[-12:]])

['tack', ' plus', 'í', 'TestResult', ' calc', 'authorize', 'El', 'Patch', 'passed', 'srs', 'xi', ' Twisted']


In [26]:
# tokenize our simple example
print(new_tokenizer(python_code).tokens())

['def', 'Ġs', 'ay', '_', 'hello', '()', 'ĊĠĠĠ', 'Ġprint', '("', 'Hello', ',', 'ĠWorld', '!")', 'Ċ', 'Ċ', '#', 'ĠPrint', 'Ġit', 'Ċ', 's', 'ay', '_', 'hello', '()', 'Ċ']


In [28]:
# check if all the python reserved keywords are in the vocab
import keyword

print(f"There are in total {len(keyword.kwlist)} Python keywords.")
for keyw in keyword.kwlist:
    if keyw not in new_tokenizer.vocab:
        print(f"No, keyword '{keyw}' is not in the vocabulary")

There are in total 35 Python keywords.
No, keyword 'await' is not in the vocabulary
No, keyword 'finally' is not in the vocabulary
No, keyword 'nonlocal' is not in the vocabulary


Let's try building a larger vocab using a larger sample of our dataset

In [29]:
length = 200000
new_tokenizer_larger = tokenizer.train_new_from_iterator(batch_iterator(), vocab_size=32768, initial_alphabet=base_vocab)

  0%|          | 0/20000 [00:00<?, ?it/s]

In [31]:
tokens = sorted(new_tokenizer_larger.vocab.items(), key=lambda x: x[1], reverse=False)
print([f'{new_tokenizer_larger.convert_tokens_to_string([t])}' for t,_ in tokens[-12:]]);

[' hexlify', 'partitioner', ' Elements', 'innerHTML', '仓', 'SomeClass', 'tlvs', ' siblings', 'hicles', 'Survey', 'ManagementClient', ' 為']


It doesn't show any regular programming keywords here, which is promising.

In [32]:
print(new_tokenizer_larger(python_code).tokens())

['def', 'Ġsay', '_', 'hello', '()', 'ĊĠĠĠ', 'Ġprint', '("', 'Hello', ',', 'ĠWorld', '!")', 'Ċ', 'Ċ', '#', 'ĠPrint', 'Ġit', 'Ċ', 'say', '_', 'hello', '()', 'Ċ']


Indents are conveniently kept. This seems more in line with our expectations of the data the model may see in the downstream task.

In [33]:
for keyw in keyword.kwlist:
    if keyw not in new_tokenizer_larger.vocab:
        print(f"No, keyword '{keyw}' is not in the vocabulary")

No, keyword 'nonlocal' is not in the vocabulary


We'll measure the performance of the tokenizer basd on the model's performance.

## Saving a Custom Tokenizer on the Hub

In [34]:
model_ckpt = "codeparrot"
org = "sergi24sanchez"
new_tokenizer_larger.push_to_hub(model_ckpt, organization=org)

/home/ssanchez/env/transformers/lib/python3.8/site-packages/transformers/utils/hub.py:731: UserWarning: The `organization` argument is deprecated and will be removed in v5 of Transformers. Set your organization directly in the `repo_id` passed instead (`repo_id={organization}/{model_id}`).
  warnings.warn(


CommitInfo(commit_url='https://huggingface.co/sergi24sanchez/codeparrot/commit/889d8522a8e499a08d4dd4a4999ce301a33b830c', commit_message='Upload tokenizer', commit_description='', oid='889d8522a8e499a08d4dd4a4999ce301a33b830c', pr_url=None, repo_url=RepoUrl('https://huggingface.co/sergi24sanchez/codeparrot', endpoint='https://huggingface.co', repo_type='model', repo_id='sergi24sanchez/codeparrot'), pr_revision=None, pr_num=None)

In [35]:
reloaded_tokenizer = AutoTokenizer.from_pretrained(org + '/' + model_ckpt)
print(reloaded_tokenizer(python_code).tokens())

tokenizer_config.json:   0%|          | 0.00/441 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

['def', 'Ġsay', '_', 'hello', '()', 'ĊĠĠĠ', 'Ġprint', '("', 'Hello', ',', 'ĠWorld', '!")', 'Ċ', 'Ċ', '#', 'ĠPrint', 'Ġit', 'Ċ', 'say', '_', 'hello', '()', 'Ċ']


In [36]:
new_tokenizer.push_to_hub(org + '/' + model_ckpt + "-small_vocabulary")

CommitInfo(commit_url='https://huggingface.co/sergi24sanchez/codeparrot-small_vocabulary/commit/8f85c7fe02327913415846bed11a88381c4e852f', commit_message='Upload tokenizer', commit_description='', oid='8f85c7fe02327913415846bed11a88381c4e852f', pr_url=None, repo_url=RepoUrl('https://huggingface.co/sergi24sanchez/codeparrot-small_vocabulary', endpoint='https://huggingface.co', repo_type='model', repo_id='sergi24sanchez/codeparrot-small_vocabulary'), pr_revision=None, pr_num=None)

# Training a Model from Scratch

Which architecture works best for the task, initialize a fresh model without pretrained weights, set up a custom data laoding class, and create a scalable training loop.

*In this section a longer thatn usual script to train a model on a distributed infrastructure will be implemented. You should not run each code snippet independently, but instead download the script provided in the [Transformers repository](https://github.com/huggingface/transformers-research-projects/tree/main/codeparrot)